In [1]:
import serial,time
from timeit import default_timer as timer
import logging
l = logging.getLogger()
l.setLevel(logging.DEBUG)

In [4]:
# initiate serial connection
def serial_init(ser):
    ser.port = "/dev/ADR_sim900"
    ser.baudrate = 9600
    ser.bytesize = serial.EIGHTBITS #number of bits per bytes
    ser.parity = serial.PARITY_NONE #set parity check: no parity
    ser.stopbits = serial.STOPBITS_ONE #number of stop bits
    ser.timeout = 0.5
    ser.write_Timeout = 0.5
    ser.xonxoff = False
    ser.rtscts= False
    ser.dsrdtr = False
    
def serial_open(ser):
    ser.open()
    time.sleep(0.2)
    ser.flushInput() #flush input buffer, discarding all its contents
    ser.flushOutput()#flush output buffer, aborting current output    

In [5]:
#get identity
def get_IDN(ser):
    ser.write('*IDN?\n')
    time.sleep(0.5)
    response = ser.readline().rstrip()
    #response = ser.read(1024).rstrip()
    return response

#get message passthrough bits
def get_RPER(ser):
    ser.write('RPER?\n')
    return ser.readline().rstrip()

#set message passthrough bits    
def set_RPER(ser, port):
    portbin = int(1 << (port))
    ser.write('RPER {}\n'.format(portbin))   

#set message passthrough bits    
def reset_RPER(ser):
    ser.write('RPER 0\n')  
    
#send command to a module  
def send_CMD_module(ser, port, cmd):
    ser.write('SNDT {},"{}"\n'.format(port, cmd))

#send command to mainframe   
def send_CMD_mf(ser, cmd):
    ser.write('{}\n'.format(cmd))   

#baud change of mainframe
def set_baud_mf(ser):
    send_CMD_mf(ser, "BAUD D,115200")
                
#baud change of a module   
def set_baud_module(ser, mod, rate):
    send_CMD_module(ser,mod,"BAUD {}".format(rate))
    send_CMD_mf(ser, "BAUD {},cmd")
    
def flush_all(ser):
    send_CMD_mf(ser, "FLOQ")
    send_CMD_mf(ser, "FLSH")
    #ser.write("*CLS\n")
    
def reset_serial_module(ser, mod):
    send_CMD_module(ser,mod,"SRST {}".format(mod))

In [6]:
#enumerate all modules
def enumerate_mods(ser):
    for i in range(1,9):
        ser.write('SNDT {},"*IDN?"\n'.format(i))
        time.sleep(0.3)
        ser.write('GETN? {},128\n'.format(i))
        time.sleep(0.3)
        # have to not use readline because empty slots return weird results otherwise
        response = ser.read(128).encode('string_escape')
        print("Slot {} - {}".format(i,response))    

## STEP 1 - INIT

In [16]:
sr = serial.Serial()
serial_init(sr)
if sr.isOpen:
    l.warn("Serial port was already open, reusing")
else:
    serial_open(sr)
print sr
sr.open()
print ("Mainframe: {}".format(get_IDN(sr)))

cRPER = get_RPER(sr)
if cRPER != '0':
    l.debug("Previous MSG config found: {} (resetting)".format(cRPER))
    reset_RPER(sr)

enumerate_mods(sr)

Serial<id=0x71256eb0, open=False>(port='/dev/ADR_sim900', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
Mainframe: 
Slot 1 - 
Slot 2 - 
Slot 3 - 
Slot 4 - 
Slot 5 - 
Slot 6 - 
Slot 7 - 
Slot 8 - 


In [20]:
sr.send_break(1)

In [15]:
sr.read(1024)

''

In [97]:
# set SIM922 to streaming output
flush_all(sr)
sr.flushInput()
set_RPER(sr, 6)
sr.read(1024)
sr.timeout = 1
send_CMD_module(sr, 6, "TVAL? 0,10")
start = timer()
for i in range(1,40):  
    response = sr.readline()
#    while (not response.startswith("MSG")): 
#        response = sr.readline()
    if (response != '\r\n'):
        response = response.rstrip()
        end = timer()
        print("Resp {}: ({}) (dt = {})".format(i,response,end - start))
        start = timer() 
    #response = sr.readline().encode('string_escape')
    #response = sr.read(1024).encode('string_escape')    
    
send_CMD_module(sr, 6, "SOUT")

Resp 1: (MSG 6,#2545.687400E+01, 4.695000E+00, 4.710000E+00, 7.940700E+01) (dt = 0.709604978561)
Resp 2: (MSG 6,#202) (dt = 0.0114789009094)
Resp 4: (MSG 6,#2545.687500E+01, 4.695000E+00, 4.714000E+00, 7.940600E+01) (dt = 0.785804986954)
Resp 5: (MSG 6,#202) (dt = 0.0117700099945)
Resp 7: (MSG 6,#2545.687100E+01, 4.695000E+00, 4.709000E+00, 7.940600E+01) (dt = 0.787793874741)
Resp 8: (MSG 6,#202) (dt = 0.0108139514923)
Resp 10: (MSG 6,#2545.687100E+01, 4.695000E+00, 4.700000E+00, 7.940500E+01) (dt = 0.786300897598)
Resp 11: (MSG 6,#202) (dt = 0.0118651390076)
Resp 13: (MSG 6,#2545.687800E+01, 4.695000E+00, 4.692000E+00, 7.940400E+01) (dt = 0.786912918091)
Resp 14: (MSG 6,#202) (dt = 0.0115160942078)
Resp 16: (MSG 6,#2545.686900E+01, 4.695000E+00, 4.688000E+00, 7.940400E+01) (dt = 0.786813020706)
Resp 17: (MSG 6,#202) (dt = 0.0120241641998)
Resp 19: (MSG 6,#2545.686900E+01, 4.695000E+00, 4.690000E+00, 7.940400E+01) (dt = 0.787047147751)
Resp 20: (MSG 6,#202) (dt = 0.0111379623413)
Resp 

In [99]:
# set SIM921 to streaming output
flush_all(sr)
sr.flushInput()
set_RPER(sr, 1)
sr.read(1024)
sr.timeout = 1.2
#send_CMD_module(sr, 1, "TPER 100")
send_CMD_module(sr, 1, "TPER 100;TVAL? 5")
start = timer()
for i in range(1,30):  
    response = sr.readline()
#    while (not response.startswith("MSG")): 
#        response = sr.readline()
    if (response != '\r\n'):
        response = response.rstrip()
        end = timer()
        print("Resp {:02d}: ({}) (dt = {})".format(i,response,end - start))
        start = timer() 
    #response = sr.readline().encode('string_escape')
    #response = sr.read(1024).encode('string_escape')    
    
send_CMD_module(sr, 1, "SOUT")

Resp 01: (MSG 1,#215+3.255582E+00) (dt = 0.114993095398)
Resp 03: (MSG 1,#215+3.259216E+00) (dt = 0.0979118347168)
Resp 05: (MSG 1,#215+3.257871E+00) (dt = 0.099417924881)
Resp 07: (MSG 1,#215+3.256008E+00) (dt = 0.0986299514771)
Resp 09: (MSG 1,#215+3.255311E+00) (dt = 0.0988380908966)
Resp 11: () (dt = 1.21767711639)
Resp 12: () (dt = 1.20133090019)
Resp 13: () (dt = 1.20133399963)
Resp 14: () (dt = 1.20109200478)
Resp 15: () (dt = 1.20135116577)
Resp 16: () (dt = 1.2013130188)
Resp 17: () (dt = 1.20132803917)
Resp 18: () (dt = 1.20139598846)
Resp 19: () (dt = 1.20136213303)
Resp 20: () (dt = 1.20135903358)
Resp 21: () (dt = 1.20133399963)
Resp 22: () (dt = 1.20757293701)
Resp 23: () (dt = 1.20133090019)
Resp 24: () (dt = 1.20135498047)
Resp 25: () (dt = 1.20138406754)
Resp 26: () (dt = 1.20132303238)
Resp 27: () (dt = 1.20133495331)
Resp 28: () (dt = 1.20013308525)
Resp 29: () (dt = 1.20063281059)


In [ ]:
send_CMD_module(sr, 6, "*IDN?")
#sr.write('GETN? 6,128\n')
time.sleep(0.5)
response = sr.readline().rstrip()
print 'R2:' + response

In [13]:
get_RPER(sr)
time.sleep(0.5)
response = sr.read(1024).rstrip()
print 'R2:' + response

R2:64


In [71]:
send_CMD_mf(sr,"RPER 64")

In [23]:
set_RPER(sr, 5, 1)
time.sleep(0.5)
response = sr.read(1024).rstrip()
print 'R2:' + response

16
R2:


In [18]:
get_IDN(sr)
time.sleep(0.5)
response = sr.read(1024).rstrip()
print 'R2:' + response

Stanford_Research_Systems,SIM900,s/n130132,ver3.6
R2:


In [73]:
sr.close()

In [100]:
print "test"

test


In [4]:
start = timer()
ser.write('*IDN?\n')
time.sleep(0.5)
response = ser.read(128)
end = timer()
print(response)
print(end - start)

Stanford_Research_Systems,SIM900,s/n130132,ver3.6

0.502631187439


In [5]:
ser.write('*IDN?\n')
time.sleep(0.5)
response = ser.readline()
print repr(response)

'Stanford_Research_Systems,SIM900,s/n130132,ver3.6\r\n'


In [ ]:
type(response)

In [6]:
string1 = 'CONN 6, "dde"\n'
ser.write(string1)

time.sleep(0.2)
response = ser.read(1024)
print 'R2:' + response

R2:


In [8]:
ser.write('*IDN?\n')
time.sleep(0.2)
response = ser.read(1024)
print 'R3:' + response

R3:Stanford_Research_Systems,SIM960,s/n014669,ver2.17



In [9]:
ser.read(1024)
maxt = maxt2 = 0
totalt = totalt2 = 0
for i in range(1,1000000):
    start = timer()
    ser.write('*IDN?\n')
    end = timer()
    #time.sleep(0.1)
    ser.timeout = 0.1
    response = ser.readline().rstrip();
    end2 = timer()
    delta = (end-start)
    delta2 = (end2-end)
    
    totalt += delta
    totalt2 += delta2
    if (maxt < delta):
        maxt = delta
    if (maxt2 < delta2):
        maxt2 = delta2
    if (response != "Stanford_Research_Systems,SIM960,s/n014669,ver2.17"):
        print("BROKE ON ITERATION {}, resp '{}'".format(i,response))
        break
    if (i % 10000 == 0):
        print("IT: {} | avg: {} | max: {} | avg2: {} | max2: {}".format(i,totalt/i,maxt,totalt2/i,maxt2))

IT: 10000 | avg: 3.29708576202e-05 | max: 0.00042200088501 | avg2: 0.0818106418371 | max2: 0.110424041748
IT: 20000 | avg: 3.20832133293e-05 | max: 0.00042200088501 | avg2: 0.0817924382091 | max2: 0.110866069794
IT: 30000 | avg: 3.20557435354e-05 | max: 0.0013439655304 | avg2: 0.0817335356633 | max2: 0.110866069794
IT: 40000 | avg: 3.22839260101e-05 | max: 0.0013439655304 | avg2: 0.0816933562756 | max2: 0.110866069794
IT: 50000 | avg: 3.2351436615e-05 | max: 0.0013439655304 | avg2: 0.0816951083469 | max2: 0.110866069794
IT: 60000 | avg: 3.21935971578e-05 | max: 0.0013439655304 | avg2: 0.081684390223 | max2: 0.110866069794
IT: 70000 | avg: 3.20450339999e-05 | max: 0.0013439655304 | avg2: 0.0816914908784 | max2: 0.110866069794
IT: 80000 | avg: 3.19592267275e-05 | max: 0.0013439655304 | avg2: 0.081686920175 | max2: 0.116405963898
IT: 90000 | avg: 3.19091055128e-05 | max: 0.0013439655304 | avg2: 0.0816924895525 | max2: 0.116405963898
IT: 100000 | avg: 3.19065928459e-05 | max: 0.00512099266

IT: 790000 | avg: 3.11742670928e-05 | max: 0.0202469825745 | avg2: 0.0816674710011 | max2: 0.11913394928
IT: 800000 | avg: 3.11616390944e-05 | max: 0.0202469825745 | avg2: 0.081667189506 | max2: 0.11913394928
IT: 810000 | avg: 3.1145173532e-05 | max: 0.0202469825745 | avg2: 0.0816654164538 | max2: 0.11913394928
IT: 820000 | avg: 3.1145084195e-05 | max: 0.0202469825745 | avg2: 0.0816656953788 | max2: 0.11913394928
IT: 830000 | avg: 3.11355898179e-05 | max: 0.0202469825745 | avg2: 0.0816639890185 | max2: 0.11913394928
IT: 840000 | avg: 3.11240116755e-05 | max: 0.0202469825745 | avg2: 0.0816652091455 | max2: 0.11913394928
IT: 850000 | avg: 3.11169242859e-05 | max: 0.0202469825745 | avg2: 0.0816650289205 | max2: 0.11913394928
IT: 860000 | avg: 3.11070134473e-05 | max: 0.0202469825745 | avg2: 0.081664653057 | max2: 0.11913394928
IT: 870000 | avg: 3.10965770963e-05 | max: 0.0202469825745 | avg2: 0.0816637724345 | max2: 0.11913394928
IT: 880000 | avg: 3.10855551199e-05 | max: 0.0202469825745 

KeyboardInterrupt: 

In [ ]:
ser.write('CURV? 0\n')
time.sleep(0.2)
response = ser.read(1024)
print 'R3:' + response

In [7]:
ser.write('TVAL? 0\n')
time.sleep(1)
response = ser.read(1024)
print 'R3:' + response

R3:5.682600E+01, 4.781000E+00, 4.790000E+00, 9.422700E+01



In [ ]:
ser.write('*RST\n')
time.sleep(0.2)
response = ser.read(1024)
print 'R3:' + response

In [ ]:
string2 = 'TVAL?' + str(4) + '\n'
ser.write(string2)

time.sleep(0.2)
response = ser.read(64)
print 'R4:' + response

In [8]:
time.sleep(0.2)
ser.write('dde\n')

response = ser.read(64)
print 'R5:' + response

R5:


In [ ]:
ser.close()

In [ ]:
ser = serial.Serial()
ser.port = "/dev/ADR_sim900"
ser.baudrate = 9600
ser.bytesize = serial.EIGHTBITS #number of bits per bytes
ser.parity = serial.PARITY_NONE #set parity check: no parity
ser.stopbits = serial.STOPBITS_ONE #number of stop bits
ser.timeout = 0.5          #non-block read
ser.write_Timeout = 0.5   #timeout for write
ser.xonxoff = False
ser.rtscts= False
ser.dsrdtr = False
ser.open()
time.sleep(2)
ser.flushInput() #flush input buffer, discarding all its contents
ser.flushOutput()#flush output buffer, aborting current output
ser.write("*RST\n")
ser.write("*CLS\n")
print('> Serial port '+ser.port+' is now open to SRS.\n Please close it when done!')
print ser

In [ ]:
string1 = '*IDN?\n'
ser.write(string1)
time.sleep(2)
response = ser.read(64)
print response

In [9]:
ser.write("*CLS\n")
ser.write("FLSH\n")
ser.timeout = 0.1
for i in range(1,9):
    ser.write('SNDT {},"*IDN?"\n'.format(i))
    time.sleep(0.1)
    ser.write('GETN? {},128\n'.format(i))
    time.sleep(0.1)
    response = ser.read(1024)
    print("Slot {} - {}".format(i,response.rstrip()))    

Slot 1 - #3051Stanford_Research_Systems,SIM921,s/n015682,ver4.2
Slot 2 - #3000
Slot 3 - #3052Stanford_Research_Systems,SIM960,s/n014669,ver2.17
Slot 4 - #3000
Slot 5 - #3051Stanford_Research_Systems,SIM925,s/n012684,ver2.4
Slot 6 - #3052Stanford_Research_Systems,SIM922,s/n012885,ver2.70
Slot 7 - #3051Stanford_Research_Systems,SIM970,s/n015635,ver2.2
Slot 8 - #3000


In [ ]:
ser.write("*CLS\n")
ser.write("FLSH\n")
ser.timeout = 0
for x in range(1000,50,-20):
    start = timer()
    ser.write('*IDN?\n')
    time.sleep(float(x)/1000)
    response = ser.read(128)
    end = timer()
    print('{} | {} | {}'.format(x,end - start,response).rstrip())

In [ ]:
ser.flushInput();
ser.write("*CLS\n")
ser.write("FLSH\n")
ser.timeout = 0.1
for x in range(1,20):    
    ser.write('*IDN?\n')
    start = timer()
    while ser.inWaiting == 0:
        pass
    end = timer()
    t1 = end - start
    response = ser.readline()
    end2 = timer()
    t2 = end2 - start
    print('{} | {} | {}'.format(t1,t2,response).rstrip())

In [ ]:
ser.write("FLSH\n")
for x in range(1000,50,-20):
    ser.write('SNDT 6,"TVAL?4"\n')
    time.sleep(float(x)/1000)
    ser.write('GETN? 6,128\n')
    time.sleep(float(x)/1000)
    response = ser.read(1024)
    ser.
    print('{} | {}'.format(x,response).rstrip())

In [ ]:
for (x * 0.1) in range(1000,50,-10):
    print(a)

In [ ]:
ser.write('SNDT 6,"TVAL?4"\n')
time.sleep(1)
ser.write('GETN? 6,128\n')
time.sleep(1)
response = ser.read(1024)
print response

In [ ]:
ser.write('TERM? 6,"TVAL?4"\n')
time.sleep(1)
ser.write('GETN? 6,128\n')
time.sleep(1)
response = ser.read(1024)
print response

In [ ]:
ser.write('SEND 6,"*RST\n"\n')
time.sleep(2)
response = ser.read(64)
print response

In [5]:
ser.write('SRST 3\n')
time.sleep(2)
response = ser.read(64)
print response

In [ ]:
ser.write('RPER?\n')
time.sleep(2)
response = ser.read(64)
print response

In [ ]:
ser.write('TERM? 6\n')
time.sleep(2)
response = ser.read(64)
print response